In [1]:
from transformers import BlipProcessor, BlipForQuestionAnswering
from PIL import Image

def blip_load():
    processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-capfilt-large",
                                              use_fast=True)
    model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-capfilt-large").to('cuda')

    n = model.num_parameters()/1000000000
    if(n<1):
        print(f'Base Model Paramters {n*1000:.0f}M')
    else:
        print(f'Base Model Paramters {n:.1f}B')

    return model,processor

def blip_inference(img_path,question_text,model,processor):
    image = Image.open(img_path)
    text = f'Answer the question in exactly one word:{question_text}'
    inputs = processor(
        image,
        text,
        return_tensors="pt",
    )
    inputs = inputs.to(model.device)
    
    out = model.generate(**inputs)
    return str(processor.decode(out[0], skip_special_tokens=True)).strip()

2025-05-01 09:59:59.614892: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746093600.046477      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746093600.163474      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model,processor = blip_load()

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/524 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.59k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

Base Model Paramters 385M


In [3]:
answer = blip_inference(
            img_path = '/kaggle/input/abo-small/images/small/00/00000529.jpg',
            question_text = "How many wheels are present?",
            model=model,
            processor=processor
        )
print(answer)

4


In [ ]:
from tqdm import tqdm
import pandas as pd

df = pd.read_csv('/kaggle/input/sqid-test-vqa/SQID_test_generated_vqa.csv')
# prepare a list to collect results
results = []

# iterate over each row, split into individual QA pairs, and predict
for _, row in tqdm(df.iterrows(), total=len(df), desc="Images"):
    img = row["image_path"]
    qs  = row["questions"].split("|")
    gts = row["answers"].split("|")

    for question, gt_answer in zip(qs, gts):

        pred = blip_inference(img_path = img,
               question_text = question,
                model=model,
                processor=processor
            )

        # store
        results.append({
            "ground_truth":  gt_answer,
            "prediction":    pred
        })

# turn your list of dicts into a DataFrame
result_df = pd.DataFrame(results)

# save it
result_df.to_csv("Blip-Baseline.csv", index=False)


Images: 100%|██████████| 984/984 [07:13<00:00,  2.27it/s]


In [5]:
pd.read_csv('/kaggle/working/Blip-Baseline.csv')

,ground_truth,prediction
0,Five,5
1,Black,black
2,Rectangular,rectangle
3,Metal,glass
4,Glasses,glasses
...,...,...
4915,Tundra,one
4916,Black,black
4917,Two,3
4918,Rectangular,rectangle
